In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager

In [6]:
driver = webdriver.Chrome()

In [7]:
main_url = 'https://www.gsmarena.com/makers.php3'

In [8]:
driver.get(main_url)

# Link

## Các thương hiệu

In [9]:
lst_webElement = driver.find_element(By.CLASS_NAME,'st-text').find_elements(By.TAG_NAME,'a')

In [12]:
lst_webElement[0]

<selenium.webdriver.remote.webelement.WebElement (session="68fe0144fd471a0f89a48bc9d71bd08b", element="0bcdfc2c-7520-4bf1-8667-b13dfc90ca72")>

In [7]:
lst_brand_phone = []
lst_link_brand = []
for br in lst_webElement:
    lst_brand_phone.append(br.text.split('\n')[0].lower())
    lst_link_brand.append(br.get_attribute('href'))

In [8]:
len(lst_brand_phone), len(lst_link_brand)

(119, 119)

In [9]:
pd.DataFrame(lst_link_brand,lst_brand_phone).to_csv('link_brand.csv')

## Các trang web sản phẩm

In [18]:
lst_link_brand = list(pd.read_csv('link_brand.csv')['0'])

In [19]:
link_all_page = []
for link in lst_link_brand:
    link_all_page.append(link)
    driver.get(link)
    driver.implicitly_wait(0.5)
    try:
        link_page = driver.find_element(By.CLASS_NAME,'nav-pages').find_elements(By.TAG_NAME,'a')
        for link_ in link_page:
            link_all_page.append(link_.get_attribute('href'))
    except NoSuchElementException:
        continue

In [20]:
len(link_all_page)

359

In [21]:
with open('link_page.text','w') as f:
    f.write('\n'.join(link_all_page))

## Link tất cả sản phẩm

In [3]:
link_all_page = []
with open('link_page.text') as f:
    link_all_page = f.readlines()
len(link_all_page)

359

In [10]:
link_all_products=[]
i=0
for link in link_all_page:
    i+=1
    driver.get(link)
#     driver.implicitly_wait(0.5)
    time.sleep(0.5)
    try:
#         print(i, end = ',')
        lst_webE_products = driver.find_element(By.CLASS_NAME,'makers').find_elements(By.TAG_NAME,'a')
#         driver.implicitly_wait(0.5)
        time.sleep(0.2)
        for item in lst_webE_products:
            link_all_products.append(item.get_attribute('href'))
    except:
        continue

In [11]:
print(len(link_all_products))

12012


In [13]:
with open('link_smartphone.txt','w') as f:
    f.write('\n'.join(link_all_products))

# Thu thập dữ liệu

In [ ]:
link_smartphone = []
with open('link_smartphone.txt') as f:
    link_smartphone = f.readlines()
len(link_smartphone)

In [ ]:
Thu thập dữ liệu bằng python

In [ ]:
lst_data = []
type_camera = {'Single':1,'Dual':2,'Triple':3,'Quad':4}

for link in link_smartphone:
    driver.get(link)
#     time.sleep(2)
#     driver.implicitly_wait(3)
    dic_tmp = dict.fromkeys(['Name','Brand','Announced','Status','Size','Resolution',
                            'OS','Chipset','Card slot','ROM','RAM','Num of MC','Pic MC','Video MC',
                            'Num of SC','Pic SC','Video SC','Loudspeaker','3.5mm jack','Battery','Price'])
    try:
        model_br = driver.find_element(By.CLASS_NAME,'specs-phone-name-title').text.split(' ')
        dic_tmp['Brand'] = model_br[0]
        dic_tmp['Name'] = ' '.join(model_br[1:])
    except NoSuchElementException:
        print('Error 429: TO MANY RQ :) in link index',i)
        break
    soup = BeautifulSoup(driver.page_source)
    soup_table = soup.find_all("table")
    for table in soup_table:

        df_tmp_raw = pd.DataFrame(pd.read_html(str(table))[0])
        df = df_tmp_raw.drop(0,axis=1).dropna(how='all').set_index(1).T
        name_table = df_tmp_raw.at[0,0].lower()
        if name_table == 'main camera':
            try:
                dic_tmp['Num of MC'] = type_camera[df.columns[0]]
                dic_tmp['Pic MC'] = ','.join(re.findall(r"(\d+) MP", df.at[2,df.columns[0]]))
                dic_tmp['Video MC'] = df.at[2,'Video']
            except:
                pass
        elif name_table == 'selfie camera':
            try:
                dic_tmp['Num of SC'] = type_camera[df.columns[0]]
                dic_tmp['Pic SC'] = ','.join(re.findall(r"(\d+) MP", df.at[2,df.columns[0]]))
                dic_tmp['Video SC'] = df.at[2,'Video']
            except:
                pass
        elif name_table == 'display':
            try:
                dic_tmp['Size'] = df.at[2,'Size']
                dic_tmp['Resolution'] = df.at[2,'Resolution']
            except:
                pass
        elif name_table == 'launch':
            try:
                dic_tmp['Announced'] = df.at[2,'Announced']
                dic_tmp['Status'] = df.at[2,'Status']
            except:
                pass
        elif name_table == 'platform':
            try:
                dic_tmp['OS'] = df.at[2,'OS']
                dic_tmp['Chipset'] = df.at[2,'Chipset']
            except:
                pass
        elif name_table == 'memory':
            try:
                dic_tmp['Card slot'] = df.at[2,'Card slot']
                romram = re.findall(r"(\d+)GB", df.at[2,'Internal'].split(', ')[0])
                rom = ram = 0
                if len(romram) == 2:
                    rom, ram = romram[0], romram[1]
                dic_tmp['ROM'], dic_tmp['RAM'] = rom, ram
            except:
                pass
        elif name_table == 'sound':
            try:
                dic_tmp['Loudspeaker'] = df.at[2,'Loudspeaker']
                dic_tmp['3.5mm jack'] = df.at[2,'3.5mm jack']
            except:
                pass
        elif name_table == 'battery':
            try:
                dic_tmp['Battery'] = re.findall(r"(\d+) mAh", df.at[2,'Type'].split(',')[0])[0]
            except:
                pass
        elif name_table == 'misc':
            try:
                dic_tmp['Price'] = df.at[2,'Price']
            except:
                pass
    lst_data.append(dic_tmp)
#     print(i, end = ',')
    i+=1

In [ ]:
len(lst_data)

In [ ]:
pd.DataFrame(lst_data).to_csv('.\data12012.csv')